# Summarise and plot IOS analysis


In [ ]:
import os
import sys
import xvec
import pickle
import textwrap
import warnings
import distinctipy
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sb
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs
import matplotlib.colors as colors
import matplotlib.ticker as ticker
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords
from _prediction import allNaN_arg

import sys
sys.path.append('/g/data/os22/chad_tmp/Aus_phenology/src')
from phenology_pixel import xr_phenometrics

%matplotlib inline

## Open per pixel phenology data

In [ ]:
lin_or_circ = 'circular'

p_ios = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/ios_analysis_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')
p_average = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_{lin_or_circ}.nc'), crs='EPSG:4326')

gdf = gpd.read_file('/g/data/os22/chad_tmp/Aus_phenology/data/vectors/IBRAv7_subregions_modified.geojson')

#seasonality mask
season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)

# sos_nan = np.isnan(p_trends['SOS_slope']).where(~np.isnan(p_average['SOS']))
# sos_nan = xr.where((sos_nan==1) & (seasonality_mask==0), 1, 0)
# eos_nan = np.isnan(p_trends['EOS_slope']).where(~np.isnan(p_average['EOS']))
# eos_nan = xr.where((eos_nan==1) & (seasonality_mask==0), 1, 0)
# pos_nan = np.isnan(p_trends['POS_slope']).where(~np.isnan(p_average['POS']))
# pos_nan = xr.where((pos_nan==1) & (seasonality_mask==0), 1, 0)
# extra_mask = (seasonality_mask | sos_nan | eos_nan | pos_nan)


In [ ]:
# cropping for masking
# crops = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/croplands_5km.nc')['croplands']
# crops = xr.where(crops==3, 0, 1) #rainfed crops
# crops = round_coords(crops)
# p_ios = p_ios.where(crops)
# p_average = p_average.where(crops)
# p_trends = p_trends.where(crops)

seasonal_vars = ['vPOS','vSOS','vEOS','SOS','POS','EOS','LOS']
# ios_most_important_var = allNaN_arg(p_ios[seasonal_vars].to_array(), dim='variable',stat='max', idx=False)
ios_most_important_var = allNaN_arg(np.abs(p_ios[seasonal_vars].to_array()), dim='variable',stat='max', idx=False)

## Partial correlations with IOS

In [ ]:
val_or_time = xr.where(ios_most_important_var<=3,0, 1).where(~np.isnan(p_average['SOS'])).where(non_seasonal!=1)

In [ ]:
vars=['vPOS', 'LOS', 'Highest partial correlation with IOS', 'Values or length']

categories = seasonal_vars
colours1 = distinctipy.get_colors(len(categories), pastel_factor=0.5, rng=2)
cmap1 = LinearSegmentedColormap.from_list("cwc", colours1, N=len(categories))

fig,axes=plt.subplots(2,2,figsize=(8,7), layout='constrained', sharey=True)

for v,ax in zip(vars, axes.ravel()):
    if v=='Highest partial correlation with IOS':
        cmap='cividis_r'
        vmin=0
        vmax=1
        t=v
        # im = ios_most_important_var.where(p_trends.vPOS_p_value <0.05).plot(ax=ax, add_colorbar=False, add_labels=False, cmap=cmap1)
        im = ios_most_important_var.where(seasonality_mask!=1).plot(ax=ax, add_colorbar=False, add_labels=False, cmap=cmap1)
        axins1 = inset_axes(ax,width="70%",height="5%",loc="lower left", borderpad=1.75)
        cbar = fig.colorbar(im, cax=axins1,  ticks=[0.4,1.3,2.2,3.1,3.9,4.8,5.7], orientation='horizontal') #ticks=[0.25,0.75]
        cbar.ax.set_xticklabels(categories, fontsize=8);

    elif v=='Values or length':
        categories = ['values', 'length']
        colours = distinctipy.get_colors(len(categories), pastel_factor=0.5, rng=2)
        cmap = LinearSegmentedColormap.from_list("cwc", colours, N=len(categories))
        t=v
        # im = ios_most_important_var.where(p_trends.vPOS_p_value <0.05).plot(ax=ax, add_colorbar=False, add_labels=False, cmap=cmap1)
        im = val_or_time.where(seasonality_mask!=1).plot(ax=ax, add_colorbar=False, add_labels=False, cmap=cmap)
        axins1 = inset_axes(ax,width="50%",height="5%",loc="lower left", borderpad=1.75)
        cbar = fig.colorbar(im, cax=axins1,  ticks=[0.25,0.75], orientation='horizontal') #ticks=[0.25,0.75]
        cbar.ax.set_xticklabels(categories, fontsize=8);
    
    else:
        cmap='plasma'
        vmin=0
        vmax=1
        t = v
        l='partial correlation w/ IOS'
        # im=p_ios[v].where(p_trends.vPOS_p_value <0.05).plot(cmap=cmap, vmin=vmin,vmax=vmax, robust=True, ax=ax, add_labels=False,add_colorbar=False)
        im=p_ios[v].where(seasonality_mask!=1).plot(cmap=cmap, vmin=vmin,vmax=vmax, robust=True, ax=ax, add_labels=False,add_colorbar=False)
        axins1 = inset_axes(ax,width="50%",height="5%",loc="lower left", borderpad=2)
        norm = colors.Normalize(vmin=vmin, vmax=vmax)
        cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        ax_cbar = fig.colorbar(cbar, ax=ax, cax=axins1, orientation='horizontal')
        ax_cbar.ax.set_title(l, fontsize=8)
    
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs='EPSG:4326', attribution='', attribution_size=1)
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    ax.set_title(t, fontsize=9)
    
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/IOS_parcorr.png',
            bbox_inches='tight', dpi=300)

In [ ]:
# ios_most_important_var.odc.explore(
#             tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
#            attr = 'Esri',
#            name = 'Esri Satellite',
#     cmap=cmap1,
# )

## Correlations across environmental gradients

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
from scipy.stats import gaussian_kde
import matplotlib.patheffects as pe

def calculate_density(x, y):
    xy = np.vstack([x, y])
    return gaussian_kde(xy)(xy)

In [ ]:
pet = xr.open_dataset('/g/data/os22/chad_tmp/Aus_phenology/data/PET_GLEAM_1982_2022.nc')['PET']
rain = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/rain_5km_monthly_1981_2022.nc')['rain']
rain = rain.sel(time=slice('1982','2022'))
pet = pet.sel(time=slice('1982','2022'))

# trees = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/trees_5km_monthly_1982_2022.nc')['trees']
# trees = trees.mean('time')

mi = rain/pet
mi = mi.rename('P:PET')
mi = xr.where(mi>5, 5, mi) #remove extreme values
mi = xr.where(mi<0, 0, mi) 

In [ ]:
mi = mi.mean('time')
mi = mi.where(seasonality_mask!=1).drop_vars('spatial_ref').to_dataframe()

mi['LOS'] = p_ios['LOS'].where(seasonality_mask!=1).drop_vars('spatial_ref').to_dataframe()
mi['vPOS'] = p_ios['vPOS'].where(seasonality_mask!=1).drop_vars('spatial_ref').to_dataframe()
# mi['trees'] = trees.where(seasonality_mask!=1).drop_vars('spatial_ref').to_dataframe()

mi = mi.dropna()
mi = mi.reset_index(drop=True)

### Trends across moisture gradient

In [ ]:
df = mi.sample(50000)

# Calculate density for LOS and vPOS plots
df["LOS_density"] = calculate_density(df["P:PET"], df["LOS"])
df["vPOS_density"] = calculate_density(df["P:PET"], df["vPOS"])

# Set up the facet grid
fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharey=True, layout='constrained')

# Plot 1: P:PET vs LOS
dff = df[["P:PET","LOS"]].rename({'P:PET':'x', 'LOS':'y'}, axis=1).reset_index(drop=True)
weights = np.polyfit(dff['x'],dff['y'], 3)
model = np.poly1d(weights)
results = smf.ols(formula='y ~ model(x)', data=dff).fit()
p1 = results.pvalues['model(x)']
r2_1 = results.rsquared

sb.scatterplot(ax=axes[0], x=df['P:PET'], y=df['LOS'], data=df, hue="LOS_density", palette="magma", alpha=0.2, s=20, linewidth=0, legend=False)
sb.lineplot(ax=axes[0], x=np.sort(dff['x']), y=model(np.sort(dff['x'])), color="blue", lw=1, path_effects=[pe.Stroke(linewidth=2, foreground='white'), pe.Normal()])

axes[0].set_title('LOS')
axes[0].set_xlabel("P:PET")
axes[0].set_ylabel("partial correlation with IOS")
axes[0].set_ylim(0,1.05)
axes[0].set_xlim(0,1.5)
axes[0].grid(alpha=0.75)
axes[0].text(0.8, 0.10, f"$p$: {p1:.3f}\n$R^2$: {r2_1:.2f}", transform=axes[0].transAxes, fontsize=10)

# Plot 2: P:PET vs vPOS
dff = df[["P:PET","vPOS"]].rename({'P:PET':'x', 'vPOS':'y'}, axis=1).reset_index(drop=True)
weights = np.polyfit(dff['x'],dff['y'], 1)
model = np.poly1d(weights)
results = smf.ols(formula='y ~ model(x)', data=dff).fit()
p2 = results.pvalues['model(x)']
r2_2 = results.rsquared

sb.scatterplot(ax=axes[1], x=df['P:PET'], y=df['vPOS'],data=df, hue="vPOS_density", palette="magma", s=20, alpha=0.2, linewidth=0, legend=False)
sb.lineplot(ax=axes[1], x=dff['x'], y=model(dff['x']), color="blue", lw=1, path_effects=[pe.Stroke(linewidth=2, foreground='white'), pe.Normal()])

axes[1].set_title('vPOS')
axes[1].set_xlabel("P:PET")
axes[1].set_ylim(0,1.05)
axes[1].grid(alpha=0.5)
axes[1].set_xlim(0,1.5)

axes[1].text(0.80, 0.85, f"$p$: {p2:.3f}\n$R^2$: {r2_2:.2f}", transform=axes[1].transAxes, fontsize=10);
fig.savefig(f'/g/data/os22/chad_tmp/Aus_phenology/results/figs/parcorr_ppet_scatterplots.png',
            bbox_inches='tight', dpi=300)